In [1]:
flex_title = "Dahsboard Transports Genevois"
flex_subtitle = "built using jupyter-flex"
flex_external_link = "https://github.com/benoitfrisque/geneva_transport_analysis"

flex_show_source = False
# flex_orientation = "rows"

In [2]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
from IPython.display import clear_output

import ipywidgets as widgets

# Montées par arrêt

## Sidebar

In [3]:
# Define the default start and end dates
slider_start_date = pd.to_datetime('2021-03-01')
slider_end_date = pd.to_datetime('2024-03-31')

slider_dates = pd.date_range(slider_start_date, slider_end_date, freq='MS')
slider_options = [(date.strftime('%B %Y'), date) for date in slider_dates]

# Create a dropdown widget for selecting the month
month_dropdown = widgets.Dropdown(
    options=slider_options,
    index=0
)

month_label = widgets.Label('Sélectionnez un mois :')

# Create a play button widget
play_button = widgets.Play(
    value=0,
    min=0,
    max=len(slider_options) - 1,
    step=1,
    interval=1500,  # Interval in milliseconds
    description="Press play",
    disabled=False
)

# Link the play button to the dropdown index
widgets.jslink((play_button, 'value'), (month_dropdown, 'index'))

# Display widgets
display(widgets.VBox([month_label, month_dropdown, play_button]))


In [4]:
# API URL
base_url = 'https://opendata.tpg.ch/api/explore/v2.1/catalog/'
endpoint = 'datasets/arrets/records'
url = base_url + endpoint

# Query parameters
offset = 0
limit = 100
select = "arretcodelong AS arret_code_long, nomarret as nom_arret, commune, pays, coordonnees, actif"
where = "" #"actif='Y'"

all_records = []

while True:
    params = {
        "limit": limit,
        "offset": offset,
        "select": select,
        "where": where
    }

    # Fetch data from the API endpoint
    response = requests.get(url, params=params)
    response = response.json()

    total_count = response['total_count']
    records = response['results']

    all_records.extend(records)

    # Increment the offset for the next request
    offset += len(records)

    # Break the loop if all records have been fetched
    if offset >= total_count:
    # if offset >=1: # for debugging
        break

# Convert all_records to a DataFrame
data_arrets = pd.json_normalize(all_records)
data_arrets.drop(columns='coordonnees', inplace=True)

data_arrets.fillna(value=np.nan, inplace=True)

data_arrets_actifs = data_arrets[data_arrets['actif'] == 'Y']

## Heatmap

### Heatmap des montées par arrêt

In [5]:
def fetch_nb_montees_descentes(start_date, end_date):
    # API URL
    base_url = 'https://opendata.tpg.ch/api/explore/v2.1/catalog/'
    endpoint = 'datasets/montees-par-arret-par-ligne/records'
    url = base_url + endpoint

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Query parameters
    offset = 0
    limit = -1
    select = "SUM(nb_de_montees) AS total_nb_de_montees, SUM(nb_de_descentes) AS total_nb_de_descentes"
    where = f"date >= '{start_date.strftime('%Y-%m-%d')}' AND date <= '{end_date.strftime('%Y-%m-%d')}'"
    group_by = "arret_code_long"
    order_by = "total_nb_de_montees DESC"

    params = {
        "select": select,
        "where": where,
        "group_by": group_by,
        "order_by": order_by,
        "limit": limit,
        "offset": offset,
    }

    # Fetch data from the API endpoint
    response = requests.get(url, params=params)
    response = response.json()

    records = response['results']

    # Convert all_records to a DataFrame
    total_montees_descentes_par_arret = pd.json_normalize(records)

    total_montees_descentes_par_arret.fillna(value=np.nan, inplace=True)

    return total_montees_descentes_par_arret


In [6]:
def plot_heatmap_nb_montees(total_montees_descentes_par_arret, start_date, end_date, out):
    with out:
        fig = px.density_mapbox(total_montees_descentes_par_arret,
                                lat='coordonnees.lat',
                                lon='coordonnees.lon',
                                z='total_nb_de_montees',
                                range_color=[0, 350e3], # fixed range for the animation
                                hover_name='arret_code_long',
                                hover_data=['nom_arret', 'commune', 'pays'],
                                radius=20,
                                center=dict(lat=46.2044, lon=6.1432),
                                zoom=11,
                                mapbox_style="open-street-map",
                                height=700,
                                title=f"Du {start_date.strftime('%d/%m/%Y')} au {end_date.strftime('%d/%m/%Y')}",
                                labels={'nom_arret':'Nom arrêt', 'total_nb_de_montees': 'Total Montees', 'arret_code_long': 'Code Arrêt', 'commune':'Commune', 'pays': 'Pays',
                                       'coordonnees.lon': 'Longitude', 'coordonnees.lat': 'Latitude'}
                               )
        
        clear_output(wait=True)
        fig.show()


In [7]:
out_heatmap = widgets.Output()
out_heatmap

Output()

## Bar chart

### Barchart des montées par arrêt

In [33]:
def plot_bar_chart_nb_montees(total_montees_descentes_par_arret, start_date, end_date, out):
    with out:
        fig = px.bar(total_montees_descentes_par_arret,
                     x='nom_arret',
                     y='total_nb_de_montees',
                     hover_name='arret_code_long',
                     hover_data=['arret_code_long'],
                     width=600,
                     height=600,
                     orientation='v',
                     color='pays',  # Color by the 'pays' column
                     color_discrete_map={'CH': 'red', 'FR': 'blue'},  # Define colors for each country
                     title=f"Du {start_date.strftime('%d/%m/%Y')} au {end_date.strftime('%d/%m/%Y')}",
                     labels={'nom_arret':'Nom arrêt', 'total_nb_de_montees': 'Total Montees', 'arret_code_long': 'Code Arrêt', 'pays': 'Pays'}
                    )

        fig.update_xaxes(categoryorder='total descending', range=[-0.5, 50.5], rangeslider_visible=True)  # Sort the bars by the total number of montees
        fig.update_yaxes(range=[0, 1.8e6], fixedrange= False)
        
        clear_output(wait=True)
        fig.show()


In [31]:
out_bar_chart = widgets.Output()
out_bar_chart

Output()

In [32]:
def fetch_and_plot(change):
    start_date = month_dropdown.value
    end_date = start_date + pd.offsets.MonthEnd(0)

    total_montees_descentes_par_arret = fetch_nb_montees_descentes(start_date, end_date)
    total_montees_descentes_par_arret = total_montees_descentes_par_arret.merge(right=data_arrets, how='left') # merge with stops metadata
    plot_bar_chart_nb_montees(total_montees_descentes_par_arret, start_date, end_date, out_bar_chart)
    plot_heatmap_nb_montees(total_montees_descentes_par_arret, start_date, end_date, out_heatmap)


month_dropdown.observe(fetch_and_plot, names="value")

fetch_and_plot(None)

# Plan des arrêts

In [12]:
def draw_map_arrets(coordonnees_centre, zoom_start=12):

    m = folium.Map(location=coordonnees_centre, zoom_start=zoom_start, min_zoom=9, control_scale=True)
    marker_cluster = MarkerCluster(name='Arrêts').add_to(m)

    for i in range(len(data_arrets_actifs)):
        lon = data_arrets_actifs.iloc[i]['coordonnees.lon']
        lat = data_arrets_actifs.iloc[i]['coordonnees.lat']

        if not np.isnan(lon) and not np.isnan(lat):
            popup_html = "<b>Nom :</b> {}<br>".format(data_arrets_actifs.iloc[i]['nom_arret'])
            popup_html += "<b>Commune :</b> {}<br>".format(data_arrets_actifs.iloc[i]['commune'])
            popup_html += "<b>Pays :</b> {}<br>".format(data_arrets_actifs.iloc[i]['pays'])
            popup_html += "<b>Code Arret :</b> {}<br>".format(data_arrets_actifs.iloc[i]['arret_code_long'])

            if data_arrets_actifs.iloc[i]['pays'] == 'CH':
                marker_color = 'red'
            else:
                marker_color = 'blue'

            folium.Marker(
                location=[lat, lon],
                tooltip=data_arrets_actifs.iloc[i]['nom_arret'],
                icon=folium.Icon(color=marker_color, icon="bus", prefix="fa"),
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(marker_cluster)

    return m

coordonnees_centre = [data_arrets_actifs['coordonnees.lat'].mean(), data_arrets_actifs['coordonnees.lon'].mean()]
m = draw_map_arrets(coordonnees_centre)
m